### The Pushshift API takes time argument as unix time, so we convert datetime to unix using datetime library

In [ ]:
import datetime
posted_after = int(datetime.datetime(2021, 11, 24).timestamp())
posted_before = int(datetime.datetime(2022, 11, 24).timestamp())

In [122]:
import json
import requests
def getData(after, before, subreddit):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?size=500&after='+str(after)+'&before='+str(before)+'&subreddit='+str(subreddit)
    #Print URL to see if the code is running properly in the specified datetime
    print(url)
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    return data['data']

### As 1 iteration of pushshift only gives 250 values, we are iterating it again my incrementing the posted_after time by 864000 which is 10 days

In [123]:
visa_data_final = []
while posted_after <= posted_before:
    visa_data = getData(posted_after,posted_before,'immigration')
    visa_data_final.extend(visa_data)
  
    posted_after += 864000  ## iterating after every 10 days

https://api.pushshift.io/reddit/search/submission/?size=500&after=1637712000&before=1669248000&subreddit=immigration
https://api.pushshift.io/reddit/search/submission/?size=500&after=1638576000&before=1669248000&subreddit=immigration
https://api.pushshift.io/reddit/search/submission/?size=500&after=1639440000&before=1669248000&subreddit=immigration
https://api.pushshift.io/reddit/search/submission/?size=500&after=1640304000&before=1669248000&subreddit=immigration
https://api.pushshift.io/reddit/search/submission/?size=500&after=1641168000&before=1669248000&subreddit=immigration
https://api.pushshift.io/reddit/search/submission/?size=500&after=1642032000&before=1669248000&subreddit=immigration
https://api.pushshift.io/reddit/search/submission/?size=500&after=1642896000&before=1669248000&subreddit=immigration
https://api.pushshift.io/reddit/search/submission/?size=500&after=1643760000&before=1669248000&subreddit=immigration
https://api.pushshift.io/reddit/search/submission/?size=500&afte

In [124]:
len(visa_data_final) ## we could fetch these many posts in last 1 year

9136

In [169]:
#visa_data_final[0:2]

In [148]:
import pandas as pd
df = pd.DataFrame(visa_data_final) ## converting json file into a dataframe
df = df[['author','url','title','score','selftext','num_comments','upvote_ratio','created_utc']]

In [149]:
## As the time created was in unix we convert it into regular datetime using this function
import datetime as dt
def proper_time(created_utc):
    return dt.datetime.fromtimestamp(created_utc)
time = df["created_utc"].apply(proper_time)
df = df.assign(post_time = time)

In [152]:
df = df.drop('created_utc', axis = 1) ## drop the old column with unix time

In [153]:
df.head()

,author,url,title,score,selftext,num_comments,upvote_ratio,post_time
0,Hydrodensity,https://www.reddit.com/r/immigration/comments/...,Looking for help with a denied I 485,1,I am frankly stumped I've been trying to figur...,15,1.0,2021-11-24 00:24:40
1,HelpfulGanache7031,https://www.reddit.com/r/immigration/comments/...,Can a green card holder immigrant apply to be ...,1,,3,1.0,2021-11-24 01:10:29
2,jamesjeffriesiii,https://www.reddit.com/r/immigration/comments/...,K-1 Visa Flexibility?,1,Hey…\n\nCan anyone tell me what’s the best Vis...,4,1.0,2021-11-24 01:29:07
3,Its_squeaks,https://www.reddit.com/r/immigration/comments/...,Should we get married before or after they are...,1,And which would be easier for immigration to t...,12,1.0,2021-11-24 02:23:51
4,mattysmidnightmusing,https://www.reddit.com/r/immigration/comments/...,My spouse came here to USA on a K1 (fiancé vis...,1,I should point out that we did in fact marry t...,14,1.0,2021-11-24 02:26:54


In [155]:
df = df.sort_values(by='post_time',ascending= False).reset_index(drop = True)

In [156]:
df.head()

,author,url,title,score,selftext,num_comments,upvote_ratio,post_time
0,hope3017,https://www.reddit.com/r/immigration/comments/...,Can I make an H1B transfer while abroad and wa...,1,This is my situation: I traveled to my home co...,0,1.0,2022-11-23 23:53:29
1,socuriouslycurious,https://www.reddit.com/r/immigration/comments/...,Australia state nomination and work experience,1,"Hi all, \n\nI hope you are all well. \n\nI hav...",0,1.0,2022-11-23 21:06:01
2,moonshine_fox21,https://www.reddit.com/r/immigration/comments/...,our CR1 timeline // may ‘21 &gt; nov ‘22 // AR...,1,hello! i half-never thought this day would com...,0,1.0,2022-11-23 20:40:16
3,headpats_required,https://www.reddit.com/r/immigration/comments/...,Leaving the US on ESTA and returning as F1.,1,"Hi, UK citizen here, gonna apply for an F1 vis...",0,1.0,2022-11-23 20:14:26
4,lesmartassaccountant,https://www.reddit.com/r/immigration/comments/...,Apply for a US Passport as a Green Card Holder,1,"Good evening everyone, \n\nI need advice in re...",0,1.0,2022-11-23 18:44:46


In [161]:
import numpy as np
## Filtering the data based on upvotes and number of comments
filtered_df = df[(df['num_comments'] > np.mean(df['num_comments'])) & (df['upvote_ratio'] > 0.6)].reset_index(drop = True)

In [162]:
filtered_df.head()

,author,url,title,score,selftext,num_comments,upvote_ratio,post_time
0,Suchu_,https://www.reddit.com/r/immigration/comments/...,H1B to H4 and then back to H1B,1,So I lost my job last week and I'm on my h1b v...,3,1.00,2022-04-25 19:43:44
1,Excellent_Froyo3552,https://www.reddit.com/r/immigration/comments/...,Tourist Visa,1,"Hi everyone, \n\nHow difficult is it for a Fil...",15,0.67,2022-04-25 18:07:00
2,National_News9532,https://www.reddit.com/r/immigration/comments/...,Deciding between US PR (Green Card) vs Canada PR,1,"Hi all,\n\nI (31M) am privileged enough to hav...",2,0.67,2022-04-25 05:08:28
3,umm__hi__,https://www.reddit.com/r/immigration/comments/...,Work Remotely?,1,"As an undergraduate student in the USA(on f1),...",2,1.00,2022-03-20 04:19:26
4,Significant-Income35,https://www.reddit.com/r/immigration/comments/...,New Permanent Resident,1,"Hi, my son filed a case for me get green card....",2,1.00,2022-03-20 02:33:56


In [170]:
filtered_df.to_csv('filtered_reddit.csv')

## Extracting comments

In [4]:
import json
import requests
def getPushshiftcomments(after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/comment/?size=500&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to see if the code is running properly in the specified datetime
    print(url)
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    return data['data']

In [5]:
comments = getPushshiftcomments(posted_after, posted_before,'immigration')

https://api.pushshift.io/reddit/search/comment/?size=500&after=1645506000&before=1669266000&subreddit=immigration


In [6]:
comments_final = []
while posted_after <= posted_before:
  ## iterating after every 10 days
  
  try:
    comments = getPushshiftcomments(posted_after,posted_before,'immigration')
    comments_final.extend(comments)
  
    posted_after += 864000
  except Exception as e:
    pass

https://api.pushshift.io/reddit/search/comment/?size=500&after=1645506000&before=1669266000&subreddit=immigration
https://api.pushshift.io/reddit/search/comment/?size=500&after=1646370000&before=1669266000&subreddit=immigration
https://api.pushshift.io/reddit/search/comment/?size=500&after=1647234000&before=1669266000&subreddit=immigration
https://api.pushshift.io/reddit/search/comment/?size=500&after=1648098000&before=1669266000&subreddit=immigration
https://api.pushshift.io/reddit/search/comment/?size=500&after=1648962000&before=1669266000&subreddit=immigration
https://api.pushshift.io/reddit/search/comment/?size=500&after=1649826000&before=1669266000&subreddit=immigration
https://api.pushshift.io/reddit/search/comment/?size=500&after=1650690000&before=1669266000&subreddit=immigration
https://api.pushshift.io/reddit/search/comment/?size=500&after=1651554000&before=1669266000&subreddit=immigration
https://api.pushshift.io/reddit/search/comment/?size=500&after=1652418000&before=1669266

In [7]:
len(comments_final)

6997

In [8]:
import pandas as pd
df_com = pd.DataFrame(comments_final)
df_com = df_com[['author','author_fullname','body','created_utc','id','link_id','parent_id','permalink','score']]

In [9]:
df_com.tail()

,author,author_fullname,body,created_utc,id,link_id,parent_id,permalink,score
6992,mukzo,t2_r8n7wxwm,The salaries in the us are relatively higher t...,1668962071,ix4347q,t3_yzyc63,t1_ix35yj1,/r/immigration/comments/yzyc63/will_graduating...,1
6993,Curios-7,t2_55wltsxj,Thanks a lot for replying. I figured the same ...,1668962328,ix43q7a,t3_yzsdgv,t1_ix2wsg6,/r/immigration/comments/yzsdgv/family_or_emplo...,1
6994,Benzolovingtraveler3,t2_8w30zdgv,Yes they will need to apply for a B2 Visa at t...,1668962742,ix44q3g,t3_y8op98,t3_y8op98,/r/immigration/comments/y8op98/do_european_gre...,1
6995,BlueNutmeg,t2_3e0ugqoy,EVERYBODY goes through AP after an interview. ...,1668963214,ix45vn0,t3_z05y5a,t3_z05y5a,/r/immigration/comments/z05y5a/need_for_hope_a...,1
6996,ANobodyWithTea,t2_dq1w2ssb,Three years would be on the very high end. 1.5...,1668964160,ix4879v,t3_z09b0c,t3_z09b0c,/r/immigration/comments/z09b0c/looking_to_get_...,1


In [10]:
#df_com.to_csv('comments.csv')